In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

# Tracking through elements with autodiff

In [2]:
import numpy as np
import torch
from torch.autograd.functional import jacobian
from torch.autograd.functional import hessian
from pytao import Tao
import matplotlib.pyplot as plt
import numdifftools as nd
from bmadx.track import *
tkwargs = {
    "dtype" : torch.double
}
import time
np.set_printoptions(precision= 16, suppress=False)
torch.set_printoptions(precision= 16, sci_mode=True)
torch.__version__, np.__version__

('1.12.1', '1.22.4')

In [4]:
from bmadx.track_a_bend import make_track_a_sbend_parts, Sbend, make_track_a_bend

In [5]:
from pathlib import Path

def find_repo(path):
    "Find repository root from the path's parents"
    for path in Path(path).parents:
        # Check whether "path/.git" exists and is a directory
        git_dir = path / ".git"
        if git_dir.is_dir():
            return path

import os
nb_dir = os.getcwd()
# Find the repo root where the script is
repo_path = str(find_repo(nb_dir))
repo_path

'/global/cfs/cdirs/m669/wlou1991/GitHub/Bmad-X'

# Constants

In [10]:
c_light = 2.99792458e8 #speed of light in m/s
m_e = 0.510998950e6 #electron mass in eV

# Bend test (body only)

In [6]:

T0 = torch.tensor(0.0, dtype=torch.float64)
def torchbend(L: torch.Tensor, P0C: torch.Tensor, G:torch.Tensor,
              DG=T0, E1=T0, E2=T0, F_INT=T0, H_GAP=T0, F_INT_X=T0, H_GAP_X=T0, FRINGE_AT="both_ends", FRINGE_TYPE="none"): 
    
    return Sbend(L=L, P0C=P0C, G=G, DG=DG, E1=E1, E2=E2, F_INT=F_INT, H_GAP=H_GAP, F_INT_X=F_INT_X, H_GAP_X=H_GAP_X, FRINGE_AT=FRINGE_AT, FRINGE_TYPE=FRINGE_TYPE)

In [7]:
# Create bend
L = 0.1 #Length in m
P0C = 4.0E+07
G = 0.5 #
DG = 0.0
E1 = 0.0
E2=0.0
F_INT = 0
H_GAP = 0
F_INT_X = 0
H_GAP_X = 0

b1 = torchbend(L=torch.tensor(L, **tkwargs), P0C=torch.tensor(P0C, **tkwargs), G=torch.tensor(G, **tkwargs),
               DG=torch.tensor(DG, **tkwargs), E1=torch.tensor(E1, **tkwargs), E2=torch.tensor(E2, **tkwargs), 
               F_INT=torch.tensor(F_INT, **tkwargs), H_GAP=torch.tensor(H_GAP, **tkwargs),
               F_INT_X=torch.tensor(F_INT_X, **tkwargs), H_GAP_X=torch.tensor(H_GAP_X, **tkwargs))

#q1 = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
#q1

b1

Sbend(L=tensor(1.0000000000000001e-01, dtype=torch.float64), P0C=tensor(4.0000000000000000e+07, dtype=torch.float64), G=tensor(5.0000000000000000e-01, dtype=torch.float64), DG=tensor(0., dtype=torch.float64), E1=tensor(0., dtype=torch.float64), E2=tensor(0., dtype=torch.float64), F_INT=tensor(0., dtype=torch.float64), H_GAP=tensor(0., dtype=torch.float64), F_INT_X=tensor(0., dtype=torch.float64), H_GAP_X=tensor(0., dtype=torch.float64), FRINGE_AT='both_ends', FRINGE_TYPE='none')

In [11]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07 #Reference particle momentum in eV
mc2 = 1*m_e # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs)
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1,ts, tp0c, tmc2)
p_in

Particle(x=tensor(2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), px=tensor(3.0000000000000001e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), y=tensor(-3.0000000000000001e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), py=tensor(-1.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), z=tensor(2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), pz=tensor(-2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), s=tensor(0., dtype=torch.float64), p0c=tensor(4.0000000000000000e+07, dtype=torch.float64), mc2=tensor( 5.1099895000000001e+05, dtype=torch.float64))

In [12]:
# create track_a_quadrupole_torch
track_a_bend_torch = make_track_a_sbend_parts(torch)[0]
# Outgoing particle
p_out = track_a_bend_torch(p_in, b1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.2932657894090434e-03, 2.8460628761045078e-03, -3.1003084927086113e-03,
        -1.0000000000000000e-03, 1.8920915439913782e-03, -2.0000000000000018e-03],
       dtype=torch.float64)

In [13]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_bend.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [14]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.2932657894089701e-03, 2.8460628761047199e-03, -3.1003084927086100e-03,
        -1.0000000000000000e-03, 1.8920915439913800e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [15]:
# close to Tao result?
torch.allclose(x_py, x_tao,  atol=1e-12, rtol = 1e-10)

True

# Bend test with hard entrance fringe

In [33]:
# Create bend
L = 0.1 #Length in m
P0C = 4.0E+07
G = 0.5 #
DG = 0.0
E1 = 0.02
E2 = 0.0
F_INT = 0.0
H_GAP = 0.00
F_INT_X = 0
H_GAP_X = 0
FRINGE_AT="entrance_end"
FRINGE_TYPE="hard_edge_only"

b1 = torchbend(L=torch.tensor(L, **tkwargs), P0C=torch.tensor(P0C, **tkwargs), G=torch.tensor(G, **tkwargs),
               DG=torch.tensor(DG, **tkwargs), E1=torch.tensor(E1, **tkwargs), E2=torch.tensor(E2, **tkwargs), 
               F_INT=torch.tensor(F_INT, **tkwargs), H_GAP=torch.tensor(H_GAP, **tkwargs),
               F_INT_X=torch.tensor(F_INT_X, **tkwargs), H_GAP_X=torch.tensor(H_GAP_X, **tkwargs),
               FRINGE_AT=FRINGE_AT, FRINGE_TYPE=FRINGE_TYPE)

#q1 = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
#q1

b1

Sbend(L=tensor(1.0000000000000001e-01, dtype=torch.float64), P0C=tensor(4.0000000000000000e+07, dtype=torch.float64), G=tensor(5.0000000000000000e-01, dtype=torch.float64), DG=tensor(0., dtype=torch.float64), E1=tensor(2.0000000000000000e-02, dtype=torch.float64), E2=tensor(0., dtype=torch.float64), F_INT=tensor(0., dtype=torch.float64), H_GAP=tensor(0., dtype=torch.float64), F_INT_X=tensor(0., dtype=torch.float64), H_GAP_X=tensor(0., dtype=torch.float64), FRINGE_AT='entrance_end', FRINGE_TYPE='hard_edge_only')

In [34]:
track_a_bend_torch = make_track_a_bend(torch)
# Outgoing particle
p_out = track_a_bend_torch(p_in, b1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.2975263173630770e-03, 2.8660060118351721e-03, -3.0968476853891649e-03,
        -9.6548477632703083e-04, 1.8919193806374857e-03, -2.0000000000000018e-03],
       dtype=torch.float64)

In [35]:
track_a_bend_torch = make_track_a_bend(torch)
# Outgoing particle
p_out = track_a_bend_torch(p_in, b1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.2975263173630770e-03, 2.8660060118351721e-03, -3.0968476853891649e-03,
        -9.6548477632703083e-04, 1.8919193806374857e-03, -2.0000000000000018e-03],
       dtype=torch.float64)

In [36]:
track_a_bend_torch = make_track_a_bend(torch)
# Outgoing particle
p_out = track_a_bend_torch(p_in, b1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.2975263173630770e-03, 2.8660060118351721e-03, -3.0968476853891649e-03,
        -9.6548477632703083e-04, 1.8919193806374857e-03, -2.0000000000000018e-03],
       dtype=torch.float64)

In [37]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_bend_hard_ent.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [38]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.2975263173629999e-03, 2.8660060118351699e-03, -3.0968476853891602e-03,
        -9.6548477632703105e-04, 1.8919329414281700e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [39]:
# close to Tao result?
torch.allclose(x_py, x_tao)

True

In [40]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=1e-12, rtol = 1e-9)

False

# Bend test with hard exit fringe

In [22]:
# Create bend
L = 0.1 #Length in m
P0C = 4.0E+07
G = 0.5 #
DG = 0.0
E1 = 0.0
E2 = 0.02
F_INT = 0.00
H_GAP = 0.0
F_INT_X = 0
H_GAP_X = 0
FRINGE_AT="exit_end"
FRINGE_TYPE="hard_edge_only"

b1 = torchbend(L=torch.tensor(L, **tkwargs), P0C=torch.tensor(P0C, **tkwargs), G=torch.tensor(G, **tkwargs),
               DG=torch.tensor(DG, **tkwargs), E1=torch.tensor(E1, **tkwargs), E2=torch.tensor(E2, **tkwargs), 
               F_INT=torch.tensor(F_INT, **tkwargs), H_GAP=torch.tensor(H_GAP, **tkwargs),
               F_INT_X=torch.tensor(F_INT_X, **tkwargs), H_GAP_X=torch.tensor(H_GAP_X, **tkwargs),
               FRINGE_AT=FRINGE_AT, FRINGE_TYPE=FRINGE_TYPE)

#q1 = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
#q1

b1

Sbend(L=tensor(1.0000000000000001e-01, dtype=torch.float64), P0C=tensor(4.0000000000000000e+07, dtype=torch.float64), G=tensor(5.0000000000000000e-01, dtype=torch.float64), DG=tensor(0., dtype=torch.float64), E1=tensor(0., dtype=torch.float64), E2=tensor(2.0000000000000000e-02, dtype=torch.float64), F_INT=tensor(0., dtype=torch.float64), H_GAP=tensor(0., dtype=torch.float64), F_INT_X=tensor(0., dtype=torch.float64), H_GAP_X=tensor(0., dtype=torch.float64), FRINGE_AT='exit_end', FRINGE_TYPE='hard_edge_only')

In [24]:
track_a_bend_torch = make_track_a_bend(torch)
# Outgoing particle
p_out = track_a_bend_torch(p_in, b1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.2908575593608778e-03, 2.8689978904183678e-03, -3.1003070675126618e-03,
        -9.7345132631462112e-04, 1.8920984408056099e-03, -2.0000000000000018e-03],
       dtype=torch.float64)

In [25]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_bend_hard_exit.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))

#tao.cmd('set ele b1 fintx = 0.8')

orbit_out = tao.orbit_at_s(ele=1)

In [15]:
#%%tao
#set ele b1 fint = 0.5
#sho ele b1

In [26]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.2908575593608102e-03, 2.8689978904185699e-03, -3.1003070675126600e-03,
        -9.7345132631462199e-04, 1.8920984393024400e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [27]:
# close to Tao result?
torch.allclose(x_py, x_tao)

True

In [32]:
# close to Tao result?
torch.allclose(x_py, x_tao,  atol=1e-12, rtol = 1e-9)

True

# Below are other non-bend tests

# Drift tests

In [4]:
# Create drift
L=1.0 # Drift length in m
d1 = Drift(L=torch.tensor(L, **tkwargs)) #named tuple is in track.py module
d1

Drift(L=tensor(1.0000000000000000e+00, dtype=torch.float64))

## Drift one particle test

In [5]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07 #Reference particle momentum in eV
mc2 = 1*m_e # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs)
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1,ts, tp0c, tmc2)
p_in

Particle(x=tensor(2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), px=tensor(3.0000000000000001e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), y=tensor(-3.0000000000000001e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), py=tensor(-1.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), z=tensor(2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), pz=tensor(-2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), s=tensor(0., dtype=torch.float64), p0c=tensor(4.0000000000000000e+07, dtype=torch.float64), mc2=tensor( 5.1099895000000001e+05, dtype=torch.float64))

In [6]:
#create track_a_drift_torch
track_a_drift_torch = make_track_a_drift(torch)
# Outgoing particle
p_out = track_a_drift_torch(p_in, d1)
x_py = torch.hstack([p_out.x,p_out.px,p_out.y,p_out.py,p_out.z,p_out.pz]).detach()
x_py

tensor([5.0060271145229325e-03, 3.0000000000000001e-03, -4.0020090381743109e-03,
        -1.0000000000000000e-03, 1.9946525738924175e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [17]:
# bmad lattice for comparison
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_drift.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out=tao.orbit_at_s(ele=1)
#orbit_out

In [18]:
# bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([5.0060271145229299e-03, 3.0000000000000001e-03, -4.0020090381743100e-03,
        -1.0000000000000000e-03, 1.9946525738923598e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [19]:
torch.allclose(x_py, x_tao)

True

## Drift Jacobian test

In [20]:
f_drift = lambda x: track_a_drift_torch(Particle(*x, ts, tp0c, tmc2), d1)[:6]
J = jacobian(f_drift, tvec1)

In [21]:
# Jacobian matrix
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.0000000000000000e+00,  1.0020180925273929e+00,
          0.0000000000000000e+00, -3.0181176940051169e-06,
          0.0000000000000000e+00, -3.0120814586171068e-03],
        [ 0.0000000000000000e+00,  1.0000000000000000e+00,
          0.0000000000000000e+00,  0.0000000000000000e+00,
          0.0000000000000000e+00,  0.0000000000000000e+00],
        [ 0.0000000000000000e+00, -3.0181176940051169e-06,
          1.0000000000000000e+00,  1.0020100442135422e+00,
          0.0000000000000000e+00,  1.0040271528723688e-03],
        [ 0.0000000000000000e+00,  0.0000000000000000e+00,
          0.0000000000000000e+00,  1.0000000000000000e+00,
          0.0000000000000000e+00,  0.0000000000000000e+00],
        [ 0.0000000000000000e+00, -3.0120814586171063e-03,
          0.0000000000000000e+00,  1.0040271528723688e-03,
          1.0000000000000000e+00,  1.7421652474771806e-04],
        [ 0.0000000000000000e+00,  0.0000000000000000e+00,
          0.0000000000000000e+00,  0.00000000000000

In [22]:
# Tao Jacobian
drift_tao = tao.matrix(0,1)
mat_tao = torch.tensor(drift_tao['mat6'], **tkwargs)
mat_tao

tensor([[ 1.0000000000000000e+00,  1.0020180925273900e+00,
          0.0000000000000000e+00, -3.0181176940051199e-06,
          0.0000000000000000e+00, -3.0120814586171098e-03],
        [ 0.0000000000000000e+00,  1.0000000000000000e+00,
          0.0000000000000000e+00,  0.0000000000000000e+00,
          0.0000000000000000e+00,  0.0000000000000000e+00],
        [ 0.0000000000000000e+00, -3.0181176940051199e-06,
          1.0000000000000000e+00,  1.0020100442135400e+00,
          0.0000000000000000e+00,  1.0040271528723699e-03],
        [ 0.0000000000000000e+00,  0.0000000000000000e+00,
          0.0000000000000000e+00,  1.0000000000000000e+00,
          0.0000000000000000e+00,  0.0000000000000000e+00],
        [ 0.0000000000000000e+00, -3.0120814586171098e-03,
          0.0000000000000000e+00,  1.0040271528723699e-03,
          1.0000000000000000e+00,  1.7421652474810300e-04],
        [ 0.0000000000000000e+00,  0.0000000000000000e+00,
          0.0000000000000000e+00,  0.00000000000000

In [23]:
# is it close to Tao result?
torch.allclose(mat_py, mat_tao)

True

# Quadrupole tests

In [114]:
def torchquadrupole(L: torch.Tensor, K1: torch.Tensor, NUM_STEPS=1,
                    X_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                    Y_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                    TILT: torch.Tensor=torch.tensor(0.0,**tkwargs)):
    return Quadrupole(L=L, K1=K1, NUM_STEPS=NUM_STEPS, X_OFFSET=X_OFFSET,
                     Y_OFFSET=Y_OFFSET, TILT=TILT)

In [115]:
# Create quad
L = 0.1 #Length in m
K1 = 10 #Quad focusing strength. Positive is focusing in x
#NUM_STEPS = 1 #number of divisions for tracking. 1 is bmad default when there are no other multipoles
q1 = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
q1

Quadrupole(L=tensor(1.0000000000000001e-01, dtype=torch.float64), K1=tensor(1.0000000000000000e+01, dtype=torch.float64), NUM_STEPS=1, X_OFFSET=tensor(0., dtype=torch.float64), Y_OFFSET=tensor(0., dtype=torch.float64), TILT=tensor(0., dtype=torch.float64))

## Quadrupole one particle test

In [116]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07 #Reference particle momentum in eV
mc2 = 1*m_e # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs) 
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1,ts, tp0c, tmc2)
p_in

Particle(x=tensor(2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), px=tensor(3.0000000000000001e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), y=tensor(-3.0000000000000001e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), py=tensor(-1.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), z=tensor(2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), pz=tensor(-2.0000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), s=tensor(0., dtype=torch.float64), p0c=tensor(4.0000000000000000e+07, dtype=torch.float64), mc2=tensor( 5.1099895000000001e+05, dtype=torch.float64))

In [117]:
# create track_a_quadrupole_torch
track_a_quadrupole_torch = make_track_a_quadrupole(torch)
# Outgoing particle
p_out = track_a_quadrupole_torch(p_in, q1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.1962397193025516e-03, 8.8418342648533682e-04, -3.2534419732692245e-03,
        -4.1008717415728594e-03, 1.9993946642253308e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [28]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [29]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.1962397193025498e-03, 8.8418342648533704e-04, -3.2534419732692201e-03,
        -4.1008717415728603e-03, 1.9993946642253299e-03, -2.0000000000000000e-03],
       dtype=torch.float64)

In [30]:
# close to Tao result?
torch.allclose(x_py, x_tao)

True

## Quadrupole Jacobian test

In [ ]:
f_quadrupole = lambda x: track_a_quadrupole_torch(Particle(*x,ts, tp0c, tmc2), q1)[:6]
J = jacobian(f_quadrupole, tvec1)

In [ ]:
# Jacobian matrix
mat_py = torch.vstack(J)
mat_py

In [ ]:
# Bmad Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'], **tkwargs)
mat_tao

In [ ]:
# close to Tao result?
torch.allclose(mat_py, mat_tao)

## Quadrupole offset test

In [ ]:
# Quad params
L = 0.1 #Length in m
K1 = 10 #Quad focusing strength. Positive is focusing in x

# quad w/o offsets
q_no_off = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
# quad with offsets
x_off = 1e-3
y_off = -2e-3
q_off = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs), 
                   X_OFFSET=torch.tensor(x_off, **tkwargs),
                   Y_OFFSET=torch.tensor(y_off, **tkwargs) )

In [ ]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07 #Reference particle momentum in eV
mc2 = 1*m_e # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs) 
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1,ts, tp0c, tmc2)
p_in

In [ ]:
# Outgoing particle no offset
p_out = track_a_quadrupole_torch(p_in, q_no_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

In [ ]:
# Outgoing particle offset
p_out = track_a_quadrupole_torch(p_in, q_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

In [ ]:
%%time
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao)

## Quadrupole tilt test (transverse rotation)

In [ ]:
# Quad params
L = 0.1 #Length in m
K1 = 10 #Quad focusing strength. Positive is focusing in x

# quad w/o offsets
q_no_off = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
# quad with offsets
tilt = 0.3
q_off = torchquadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs),
                        TILT = torch.tensor(tilt, **tkwargs))

In [ ]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07 #Reference particle momentum in eV
mc2 = 1*m_e # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs) 
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1,ts, tp0c, tmc2)
p_in

In [ ]:
# Outgoing particle no offset
p_out = track_a_quadrupole_torch(p_in, q_no_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

In [ ]:
# Outgoing particle offset
p_out = track_a_quadrupole_torch(Particle(*tvec1,ts, tp0c, tmc2), q_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

In [ ]:
%%time
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao)

# Lattice tracking

## Lattice one particle test

In [ ]:
# Create drift
L_d = 1.0 # Drift length in m
d1 = Drift(torch.tensor(L_d, **tkwargs))
# Create quad
L_q = 0.1  # quad length in m
K1 = 10  # Quad focusing strength. Positive is focusing in x
#NUM_STEPS = 1  # number of divisions for tracking. 1 is bmad default when there are no other multipoles
q1 = torchquadrupole(L=torch.tensor(L_q, **tkwargs), K1=torch.tensor(K1, **tkwargs))

In [ ]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07  # Reference particle momentum in eV
mc2 = 1*m_e  # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs)
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1, ts, tp0c, tmc2)
p_in

In [ ]:
# Lattice example
lattice = [d1, q1, d1, q1, d1]  # lattice is a list of elements
# List of particle coordinates after each element:
x_list = [torch.hstack(coords[:6]).detach() for coords in track_a_lattice(p_in, lattice)]
# Outgoing particle after complete lattice:
x_py = torch.hstack(track_a_lattice(p_in, lattice)[-1][:6]).detach()
# alternative: x_list[-1]
x_py

In [ ]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_drift_quad.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=5)

In [ ]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

In [ ]:
# close to Tao result?
torch.allclose(x_py, x_tao)

## Lattice Jacobian test

In [ ]:
f_driftquadrupole = lambda x: track_a_lattice(Particle(*x, ts, tp0c, tmc2), lattice)[-1][:6]
J = jacobian(f_driftquadrupole, tvec1)

In [ ]:
# Jacobian matrix
mat_py = torch.vstack(J)
mat_py

In [ ]:
# Bmad Jacobian
lat_tao = tao.matrix(0,5)
mat_tao = torch.tensor(lat_tao['mat6'], **tkwargs)
mat_tao

In [ ]:
# close to Tao result?
torch.allclose(mat_py, mat_tao)

## Multi-particle propagation through lattice

In [ ]:
# Particle bunch with Gaussian distribution
sample_size = 1000
mean = torch.zeros(6, **tkwargs)
cov = torch.diag(torch.tensor([1e-6, 2e-6, 1e-6, 2e-6, 1e-6, 2e-6],**tkwargs))
dist = torch.distributions.multivariate_normal.MultivariateNormal(mean, cov)
sample = dist.sample(torch.Size([sample_size]))
p_in = Particle(*sample.T, ts, tp0c, tmc2)

In [ ]:
p_out = track_a_lattice(p_in, lattice)

In [ ]:
# some beam properties up and downstream
torch.std(p_out[0].y),torch.std(p_out[-1].y)

## Stub element test

In [ ]:
# divide a quad into 10 parts
divided_quad = stub_element(q1, 10)
divided_quad

## Plotting using stub_element

In [ ]:
# make a quadrupole triplet
L_d = 1.5  # Drift length in m
d1 = Drift(torch.tensor(L_d, **tkwargs)) # drift
L_q = 0.1  # Quadrupole length in m
K1 = 10  # Quadrupole strengths
NUM_STEPS = 1
q1 = torchquadrupole(L=torch.tensor(L_q,**tkwargs), K1=torch.tensor(K1, **tkwargs))  # x-focusing
q2 = torchquadrupole(L=torch.tensor(L_q,**tkwargs), K1=torch.tensor(-K1, **tkwargs))  # y-focusing
lattice = [d1, q1, d1, q2, d1, q1, d1]

# stub each element into n equal parts each
n=50
stubbed_lattice = stub_lattice(lattice, n)

In [ ]:
all_p = track_a_lattice(p_in, stubbed_lattice)
stdx = np.array([torch.std(par.x).item() for par in all_p])
stdy = np.array([torch.std(par.y).item() for par in all_p])
s = np.array([par.s.item() for par in all_p])
plt.plot(s, stdx*1000, label=r'$\sigma_x$')
plt.plot(s, stdy*1000, label=r'$\sigma_y$')
plt.xlabel(r'$s$ (m)')
plt.ylabel(r'$\sigma_{x,y}$ (mm)')
plt.legend()

# Hessian Matrix example

In [ ]:
# Particle bunch with Gaussian distribution
sample_size = 1000
mean = torch.zeros(6, **tkwargs)
cov = torch.diag(torch.tensor([1e-6, 2e-6, 1e-6, 2e-6, 1e-6, 2e-6],**tkwargs))
dist = torch.distributions.multivariate_normal.MultivariateNormal(mean, cov)
sample = dist.sample(torch.Size([sample_size]))
p_in = Particle(*sample.T, ts, tp0c, tmc2)

L_d = 1.00 # Drift length
L_q = 0.1 # Quad length 
drift = Drift(torch.tensor(L_d, **tkwargs))

def sigmax_end(k1s):
    """returns x beamsize after lattice composed by len(k1s)+1 
    drifts with len(k1s) quadrupoles in between.
    """
    lattice = [drift]
    
    for k1 in k1s:
        lattice.append(torchquadrupole(L=torch.tensor(L_q, **tkwargs), K1=k1))
        lattice.append(drift)

    p_out = track_a_lattice(p_in, lattice)[-1]
    return torch.std(p_out.x)

#k1s = torch.tensor([10,-10,10,-10,10,-10,10,-10,10,-10], **tkwargs)
k1s = torch.zeros(10, **tkwargs)

In [ ]:
#Hessian using autodiff
%time
hessian_py = hessian(sigmax_end,k1s)
hessian_py

In [ ]:
plt.imshow(hessian_py.detach().numpy())
plt.colorbar()
#plt.savefig("hessian.eps")

In [ ]:
#Hessian using numerical differentiation

p_in = Particle(*sample.detach().numpy().T, 0, p0c, mc2)

L_d = 1.00 # Drift length
L_q = 0.1 # Quad length 
drift = Drift(L_d)
def sigmax_end2(k1s):
    """returns x beamsize after lattice composed by len(k1s)+1 
    drifts with len(k1s) quadrupoles in between.
    """
    lattice = [drift]
    
    for k1 in k1s:
        lattice.append(Quadrupole(L=L_q, K1=k1))
        lattice.append(drift)

    p_out = track_a_lattice(p_in, lattice)[-1]
    
    return np.std(p_out.x)

#k1s = np.array([10,-10,10,-10,10,-10,10,-10,10,-10])
k1s = np.zeros(10)

In [ ]:
%time
hessian_nd = nd.Hessian(sigmax_end2)(k1s)
hessian_nd

In [ ]:
plt.imshow(hessian_nd)
plt.colorbar()

In [ ]:
np.allclose(hessian_nd, hessian_py.detach().numpy())

In [ ]:
fig, axs = plt.subplots(2, figsize=(10,10))
cm = axs[0].imshow(hessian_py.detach().numpy())
fig.colorbar(cm,ax=axs[0])
#ax.imshow(hessian_py.detach().numpy())
#ax = axs[1]
#ax.imshow(1-hessian_nd)
cm = axs[1].imshow(hessian_nd)
fig.colorbar(cm,ax=axs[1])

In [ ]:
hessian_numpy = hessian_py.detach().numpy()
for i in range(len(hessian_nd[:,0])):
    hessian_numpy[i,i]=0
fig, axs = plt.subplots(2, figsize=(10,10))
cm = axs[0].imshow(hessian_numpy)
fig.colorbar(cm,ax=axs[0])
#ax.imshow(hessian_py.detach().numpy())
#ax = axs[1]
#ax.imshow(1-hessian_nd)
cm = axs[1].imshow(hessian_nd)
fig.colorbar(cm,ax=axs[1])